In [ ]:
#in vs code terminal, create conda environment with python: conda create --name *myenv* python=3.9
#pip install numpy==1.26.4 opencv-python==4.7.0.72 mediapipe==0.10.21
#make sure kernel is opt/anaconda3/envs/try/bin/python
import sys
print(sys.executable)

In [ ]:
#test cell, ensure camera permissions are on
import numpy as np
import cv2 as cv
 
cap = cv.VideoCapture(0)
if not cap.isOpened():
    print("Cannot open camera")
    exit()
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
 
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    # Our operations on the frame come here
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    # Display the resulting frame
    cv.imshow('frame', gray)
    if cv.waitKey(1) == ord('q'): #press q when on camera capture window only
        break
 
# When everything done, release the capture
cap.release()
cv.destroyAllWindows()

In [1]:
#analyze dog breeds through static images to create a dictionary of dog breed features
import cv2 as cv
import mediapipe as mp
import numpy as np
import os

#initialize mediapipe face mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=True,   #turn on still images
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5
)

#define image folder in correct directiry
current_directory = os.getcwd()
images_path = os.path.join(current_directory, "images")
#check that program is accessing the correct images folder
print(images_path)
image_files = [f for f in os.listdir(images_path) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]

#dictionary for dog breeds and their facial features
dogs = {}

for img_file in image_files:
    img_path = os.path.join(images_path, img_file)
    image = cv.imread(img_path)
    if image is None:
        print(f"Could not read image {img_file}, skipping.")
        continue

    h, w, _ = image.shape
    rgb_image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    result = face_mesh.process(rgb_image)

    if result.multi_face_landmarks:
        for face_landmarks in result.multi_face_landmarks:
            #convert landmark coordinates to pixel positions
            points = np.array([(int(lm.x * w), int(lm.y * h)) for lm in face_landmarks.landmark])

            #face size (height/width)
            x_min, y_min = np.min(points, axis=0)
            x_max, y_max = np.max(points, axis=0)
            face_width = x_max - x_min
            face_height = y_max - y_min
            face_ratio = face_height / face_width if face_width != 0 else 0

            #eye distance
            left_eye_idx = 33
            right_eye_idx = 263
            eye_dist = np.linalg.norm(np.array(points[left_eye_idx]) - np.array(points[right_eye_idx]))

            #nose ratio
            nose_tip_idx = 1
            nose_bottom_idx = 2
            nose_left_idx = 98
            nose_right_idx = 327
            nose_length = np.linalg.norm(np.array(points[nose_tip_idx]) - np.array(points[nose_bottom_idx]))
            nose_width = np.linalg.norm(np.array(points[nose_left_idx]) - np.array(points[nose_right_idx]))
            nose_ratio = nose_length / nose_width if nose_width != 0 else 0

            #eye color
            l_eye_center = points[468]
            r_eye_center = points[473]
            patch_size = 3
            l_patch = image[l_eye_center[1]-patch_size:l_eye_center[1]+patch_size,
                            l_eye_center[0]-patch_size:l_eye_center[0]+patch_size]
            r_patch = image[r_eye_center[1]-patch_size:r_eye_center[1]+patch_size,
                            r_eye_center[0]-patch_size:r_eye_center[0]+patch_size]
            eye_color = None
            if l_patch.size > 0 and r_patch.size > 0:
                eye_color = np.mean(np.vstack((l_patch.reshape(-1, 3), r_patch.reshape(-1, 3))), axis=0).astype(int)

            #skin color
            face_roi = image[y_min:y_max, x_min:x_max]
            skin_color = None
            if face_roi.size > 0:
                skin_color = np.mean(face_roi.reshape(-1, 3), axis=0).astype(int)
            
            #store results in dogs dictionary
            breed = os.path.splitext(img_file)[0]
            breed = breed.replace("_", " ")
            dogs[breed] = {
                "face ratio": round(face_ratio, 2),
                "eye distance": round(eye_dist, 1),
                "nose ratio": round(nose_ratio, 2),
                "eye color": eye_color.tolist() if eye_color is not None else None,
                "skin color": skin_color.tolist() if skin_color is not None else None
            }

    else:
        print(f"No face detected in {img_file}")
        dogs[img_file] = None

for k, v in dogs.items():
    print(f"{k}: {v}")


I0000 00:00:1762309798.300369  262013 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1762309798.317744  262120 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762309798.328529  262120 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762309798.351226  262119 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


/Users/KaylaNguyen/Desktop/try/images
Goldendoodle: {'face ratio': 1.04, 'eye distance': 422.9, 'nose ratio': 0.16, 'eye color': [48, 56, 67], 'skin color': [101, 136, 182]}
German Shepherd: {'face ratio': 1.14, 'eye distance': 132.0, 'nose ratio': 0.2, 'eye color': [81, 78, 79], 'skin color': [49, 68, 97]}
Siberian Husky: {'face ratio': 1.13, 'eye distance': 299.0, 'nose ratio': 0.21, 'eye color': [175, 139, 86], 'skin color': [148, 149, 148]}
French Bulldog: {'face ratio': 1.0, 'eye distance': 1178.1, 'nose ratio': 0.15, 'eye color': [14, 10, 13], 'skin color': [79, 100, 119]}
Chihuahua: {'face ratio': 1.02, 'eye distance': 243.2, 'nose ratio': 0.24, 'eye color': [71, 67, 75], 'skin color': [130, 150, 179]}
Shiba: {'face ratio': 1.13, 'eye distance': 195.0, 'nose ratio': 0.28, 'eye color': [31, 38, 50], 'skin color': [136, 168, 205]}
Maltipoo: {'face ratio': 1.03, 'eye distance': 176.1, 'nose ratio': 0.28, 'eye color': [26, 24, 24], 'skin color': [153, 163, 171]}
Golden Retreiver: {'

In [2]:
#main code 
import numpy as np
import cv2 as cv
import mediapipe as mp
 
CSS3_COLORS = {
    "AntiqueWhite": "#FAEBD7",
    "AliceBlue": "#F0F8FF",
    "Aqua": "#00FFFF",
    "Aquamarine": "#7FFFD4",
    "Azure": "#F0FFFF",
    "Beige": "#F5F5DC",
    "Bisque": "#FFE4C4",
    "Black": "#000000",
    "BlanchedAlmond": "#FFEBCD",
    "Brown": "#A52A2A",
    "BurlyWood": "#DEB887",
    "CadetBlue": "#5F9EA0",
    "Chocolate": "#D2691E",
    "Coral": "#FF7F50",
    "CornflowerBlue": "#6495ED",
    "Cornsilk": "#FFF8DC",
    "DarkGoldenRod": "#B8860B",
    "DarkGray": "#A9A9A9",
    "DarkGreen": "#006400",
    "DarkKhaki": "#BDB76B",
    "DarkOliveGreen": "#556B2F",
    "DarkSlateGray": "#2F4F4F",
    "DeepSkyBlue": "#00BFFF",
    "GhostWhite": "#F8F8FF",
    "GoldenRod": "#DAA520",
    "LemonChiffon": "#FFFACD",
    "LightBlue": "#ADD8E6",
    "LightGoldenRodYellow": "#FAFAD2",
    "LightSkyBlue": "#87CEFA",
    "LightSteelBlue": "#B0C4DE",
    "Moccasin": "#FFE4B5",
    "NavajoWhite": "#FFDEAD",
    "Orange": "#FFA500",
    "Peru": "#CD853F",
    "SaddleBrown": "#8B4513",
    "SandyBrown": "#F4A460",
    "Tan": "#D2B48C",
    "Aqua": "#00FFFF"
}

def color_distance(c1, c2):
    if c1 is None or c2 is None:
        return 0
    return np.linalg.norm(np.array(c1) - np.array(c2))

def feature_distance(human, dog):
    dist = 0
    dist += (human["face_ratio"] - dog["face ratio"])**2
    dist += (human["eye distance"] - dog["eye distance"])**2
    dist += (human["nose ratio"] - dog["nose ratio"])**2
    dist += color_distance(human["eye color"], dog["eye color"])**2
    dist += color_distance(human["skin color"], dog["skin color"])**2
    return np.sqrt(dist)

def find_closest_breed(human, dogs):
    best_breed = None
    min_dist = float('inf')
    for breed, features in dogs.items():
        if features is None:
            continue
        dist = feature_distance(human, features)
        if dist < min_dist:
            min_dist = dist
            best_breed = breed
    return best_breed, min_dist

#convert hex string to rgb tuple
def hex_to_rgb(hex_str):
    hex_str = hex_str.lstrip("#")
    return tuple(int(hex_str[i:i+2], 16) for i in (0, 2, 4))

#helper function to get color names
def color_name(bgr):
    rgb = bgr[::-1]
    min_dist = float('inf')
    closest_name = "Unknown"
    for name, hex_val in CSS3_COLORS.items():
        r_val, g_val, b_val = hex_to_rgb(hex_val)
        dist = (r_val - rgb[0])**2 + (g_val - rgb[1])**2 + (b_val - rgb[2])**2
        if dist < min_dist:
            min_dist = dist
            closest_name = name
    return closest_name


#initialize mediapipe face mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

#lists to store face measurements
face_ratios = []
eye_dists = []
nose_ratios = []
eye_colors = []
skin_colors = []

#number of frames to average for output
N = 30

#previous facial detection values for comparison
prev_face_ratio = None
prev_eye_dist = None
prev_nose_ratio = None
prev_eye_color = None
prev_skin_color = None

#thresholds for facial difference
FACE_RATIO_THRESH = 0.05
EYE_DIST_THRESH = 10 #pixel
NOSE_RATIO_THRESH = 0.05
COLOR_THRESH = 3.0  #euclidean dist in bgr space

#if color change is over threshold then return
def color_change(c1, c2):
    if c1 is None or c2 is None:
        return True
    return np.linalg.norm(c1 - c2) > COLOR_THRESH

#start video capture
cap = cv.VideoCapture(0)
if not cap.isOpened():
    print("Cannot open camera")
    exit()

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
 
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    
    #convert to RGB for mediapipe processing
    rgb_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    result = face_mesh.process(rgb_frame)

    #convert back to BGR for opencv 
    frame = cv.cvtColor(rgb_frame, cv.COLOR_RGB2BGR)

    h, w, _ = frame.shape

    if result.multi_face_landmarks:
        for face_landmarks in result.multi_face_landmarks:
            #convert landmark coordinates to pixel positions
            points = np.array([(int(lm.x * w), int(lm.y * h)) for lm in face_landmarks.landmark])

            #face size (height/width)
            x_min, y_min = np.min(points, axis=0)
            x_max, y_max = np.max(points, axis=0)
            face_width = x_max - x_min
            face_height = y_max - y_min
            face_ratio = face_height / face_width if face_width != 0 else 0

            #eye distance
            left_eye_idx = 33
            right_eye_idx = 263
            eye_dist = np.linalg.norm(np.array(points[left_eye_idx]) - np.array(points[right_eye_idx]))

            #nose size
            nose_tip_idx = 1
            nose_bottom_idx = 2
            nose_left_idx = 98
            nose_right_idx = 327
            nose_length = np.linalg.norm(np.array(points[nose_tip_idx]) - np.array(points[nose_bottom_idx]))
            nose_width = np.linalg.norm(np.array(points[nose_left_idx]) - np.array(points[nose_right_idx]))
            nose_ratio = nose_length / nose_width if nose_width != 0 else 0

            #eye color
            l_eye_center = points[468]
            r_eye_center = points[473]
            patch_size = 3
            l_patch = frame[l_eye_center[1]-patch_size:l_eye_center[1]+patch_size,
                            l_eye_center[0]-patch_size:l_eye_center[0]+patch_size]
            r_patch = frame[r_eye_center[1]-patch_size:r_eye_center[1]+patch_size,
                            r_eye_center[0]-patch_size:r_eye_center[0]+patch_size]
            eye_color = None
            if l_patch.size > 0 and r_patch.size > 0:
                eye_color = np.mean(np.vstack((l_patch.reshape(-1, 3), r_patch.reshape(-1, 3))), axis=0).astype(int)

            #skin color
            face_roi = frame[y_min:y_max, x_min:x_max]
            skin_color = None
            if face_roi.size > 0:
                skin_color = np.mean(face_roi.reshape(-1, 3), axis=0).astype(int)

            #append calculations to lists for averaging
            face_ratios.append(face_ratio)
            eye_dists.append(eye_dist)
            nose_ratios.append(nose_ratio)
            if eye_color is not None:
                eye_colors.append(eye_color)
            if skin_color is not None:
                skin_colors.append(skin_color)

            #compute averages
            if len(face_ratios) >= N:
                avg_face_ratio = sum(face_ratios[-N:]) / N
                avg_eye_distance = sum(eye_dists[-N:]) / N
                avg_nose_ratio = sum(nose_ratios[-N:]) / N
                avg_eye_color = np.mean(np.array(eye_colors[-N:]), axis=0).astype(int)
                avg_skin_color = np.mean(np.array(skin_colors[-N:]), axis=0).astype(int)
                
                #calculate if change in facial detection is over threshold
                change = (
                    prev_face_ratio is None or 
                    abs(avg_face_ratio - prev_face_ratio) > FACE_RATIO_THRESH or
                    abs(avg_eye_distance - prev_eye_dist) > EYE_DIST_THRESH or
                    abs(avg_nose_ratio - prev_nose_ratio) > NOSE_RATIO_THRESH or
                    color_change(avg_eye_color, prev_eye_color) or
                    color_change(avg_skin_color, prev_skin_color)
                )

                #if change is over threshold, change color names and output
                if change:
                    eye_color_name = color_name(avg_eye_color)
                    skin_color_name = color_name(avg_skin_color)

                    #print output
                    human = {
                        "face_ratio": avg_face_ratio,
                        "eye distance": avg_eye_distance,
                        "nose ratio": avg_nose_ratio,
                        "eye color": avg_eye_color,
                        "skin color": avg_skin_color,
                    }
                    print("Face ratio: {:.2f}".format(avg_face_ratio))
                    print("Eye distance: {:.1f}px".format(avg_eye_distance))
                    print("Nose ratio: {:.2f}".format(avg_nose_ratio))
                    print("Eye color:", eye_color_name, "(BGR: {})".format(avg_eye_color))
                    print("Skin color:", skin_color_name, "(BGR: {})".format(avg_skin_color))
                    print("-"*50)

                    closest_breed, distance = find_closest_breed(human, dogs)
                    print(f"Closest dog breed: {closest_breed} (distance = {distance:.2f})")

                    #update previous values
                    prev_face_ratio = avg_face_ratio
                    prev_eye_dist = avg_eye_distance
                    prev_nose_ratio = avg_nose_ratio
                    prev_eye_color = avg_eye_color
                    prev_skin_color = avg_skin_color

                # clear lists to restart averaging
                face_ratios.clear()
                eye_dists.clear()
                nose_ratios.clear()
                eye_colors.clear()
                skin_colors.clear()

    
    # Display the resulting frame
    cv.imshow('frame', frame)
    if cv.waitKey(1) == ord('q'): #press q when on camera capture window only
        break
 
# When everything done, release the capture
cap.release()
cv.destroyAllWindows()

I0000 00:00:1762309804.254397  262013 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M2
W0000 00:00:1762309804.256580  262204 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1762309804.266501  262202 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Face ratio: 1.20
Eye distance: 279.9px
Nose ratio: 0.20
Eye color: Black (BGR: [14 12 28])
Skin color: Peru (BGR: [ 94 116 170])
--------------------------------------------------
Closestdog breed: Chihuahua (distance = 111.20)
Face ratio: 1.20
Eye distance: 279.9px
Nose ratio: 0.20
Eye color: Black (BGR: [16 12 27])
Skin color: Peru (BGR: [104 119 176])
--------------------------------------------------
Closestdog breed: Chihuahua (distance = 106.52)
Face ratio: 1.20
Eye distance: 278.8px
Nose ratio: 0.21
Eye color: Black (BGR: [22 19 39])
Skin color: Peru (BGR: [104 119 176])
--------------------------------------------------
Closestdog breed: Chihuahua (distance = 94.41)
Face ratio: 1.20
Eye distance: 280.8px
Nose ratio: 0.20
Eye color: Black (BGR: [15 12 28])
Skin color: Peru (BGR: [104 119 176])
--------------------------------------------------
Closestdog breed: Chihuahua (distance = 106.92)
